In [92]:
import os

os.chdir("..")

In [ ]:
# contrtller.py
import tkinter as tk
from tkinter import ttk, messagebox
import random
import json
import os
# from app.vocabulary_builder.model.vocabulary_model import VocabularyModel
# from app.vocabulary_builder.view.vocabulary_view import VocabularyView  # Remova esta importação

# model.py
from tkinter import messagebox
import random
import json
import os

class VocabularyModel:
    def __init__(self, data_folder="data"):
        self.data_folder = data_folder
        self.current_category = None
        self.current_subcategory = None
        self.words = {}
        self.current_word = None
        self.current_answer = None

    def list_categories(self):
        categories = [d for d in os.listdir(self.data_folder) if os.path.isdir(os.path.join(self.data_folder, d))]
        return categories

    def list_subcategories(self, category):
        category_path = os.path.join(self.data_folder, category)
        if os.path.exists(category_path) and os.path.isdir(category_path):
            subcategories = [f[:-5] for f in os.listdir(category_path) if f.endswith(".json")]
            return subcategories
        return []

    def load_vocabulary(self, category, subcategory):
        filepath = os.path.join(self.data_folder, category, f"{subcategory}.json")
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                self.words = json.load(f)
            return True
        except FileNotFoundError:
            messagebox.showerror("Error", f"Vocabulary file not found: {filepath}")
            return False
        except json.JSONDecodeError:
            messagebox.showerror("Error", f"Error decoding JSON in: {filepath}")
            return False

    def get_next_word(self):
        if not self.words:
            return None, None
        self.current_word = random.choice(list(self.words.keys()))
        self.current_answer = self.words[self.current_word]
        return self.current_word, self.current_answer

    def check_answer(self, user_answer):
        if self.current_word and user_answer.strip().lower() == self.current_answer.lower():
            self.words.pop(self.current_word)
            return True
        return False
    


In [121]:
vocabulary_model = VocabularyModel()
vocabulary_model.load_vocabulary(category="animals", subcategory="birds")

True

In [120]:
vocabulary_model.list_subcategories("animals")

['birds', 'mammals']

In [129]:
vocabulary_model.get_next_word()

('parrot', 'papagaio')

### Tela do vocabulario

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import ttk
import os

class Pagina(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller

    def centralizar_widget(self, widget, rely):
        widget.place(relx=0.5, rely=rely, anchor=tk.CENTER)

class PaginaInicial(Pagina):
    def __init__(self, parent, controller):
        Pagina.__init__(self, parent, controller)

        label = tk.Label(self, text="Esta é a Página Inicial")
        self.centralizar_widget(label, 0.2)
        botao1 = tk.Button(self, text="Jogo Vocabulario", command=lambda: controller.mostrar_frame(PageGame))
        self.centralizar_widget(botao1, 0.5)
        botao2 = tk.Button(self, text="Ir para a Página 2", command=lambda: controller.mostrar_frame(Pagina2))
        botao2.place(relx=0.7, rely=0.5, anchor=tk.CENTER) # Exemplo sem centralizar

class PaginaComBotaoVoltar(Pagina):
    def __init__(self, parent, controller, texto_label):
        Pagina.__init__(self, parent, controller)
        label = tk.Label(self, text=texto_label)
        self.centralizar_widget(label, 0.2)
        botao_voltar = tk.Button(self, text="Voltar para a Inicial", command=lambda: controller.mostrar_frame(PaginaInicial))
        self.centralizar_widget(botao_voltar, 0.8)

class Pagina2(PaginaComBotaoVoltar):
    def __init__(self, parent, controller):
        super().__init__(parent, controller, "Esta é a Página 2")

class PageGame(Pagina):
    def __init__(self, parent, controller):
        Pagina.__init__(self, parent, controller)
        self.controller = controller  # Garante que o controller esteja acessível

        self.vocabulary_model:VocabularyModel = controller.vocabulary_model
        
        label_title = tk.Label(self, text="Seleção do Tipo de Jogo", font=("Arial", 16))
        self.centralizar_widget(label_title, 0.1)

        label_category = tk.Label(self, text="Escolha uma categoria:", font=("Arial", 14))
        self.centralizar_widget(label_category, 0.2)

        self.category_combobox = ttk.Combobox(self, state="readonly")
        self.centralizar_widget(self.category_combobox, 0.3)
        self.category_combobox.bind("<<ComboboxSelected>>", self.populate_subcategories)

        label_subcategory = tk.Label(self, text="Escolha uma subcategoria:", font=("Arial", 14))
        self.centralizar_widget(label_subcategory, 0.4)

        self.subcategory_combobox = ttk.Combobox(self, state="readonly")
        self.centralizar_widget(self.subcategory_combobox, 0.5)
        self.subcategory_combobox["state"] = "disabled"  # Inicialmente desabilitado

        start_button = tk.Button(self, text="Iniciar Jogo", command=self.start_game, font=("Arial", 12), state="disabled")
        self.centralizar_widget(start_button, 0.7)
        self.start_button = start_button  # Salva referência para habilitar depois

        # Dados de exemplo para categorias (substitua pela sua lógica real)
        # self.categories = self.controller.get_categories()# ["Matemática", "Português", "Inglês", "História"]
        # self.categories = ["Matemática", "Português", "Inglês", "História"]
        # self.category_combobox["values"] = self.categories

        self.populate_categories()
    
    def populate_categories(self):
        self.category_combobox["values"] = self.vocabulary_model.get_categories()

    def populate_subcategories(self, event):
        category = self.category_combobox.get()
        subcategories = self.vocabulary_model.get_subcategories(category) if hasattr(self.vocabulary_model, 'get_subcategories') else []
        print(category, subcategories,self.vocabulary_model, hasattr(self.vocabulary_model, 'get_subcategories'), self.vocabulary_model.get_subcategories(category))
        
        self.vocabulary_model.load_vocabulary(category=category, subcategory=subcategories)
        self.subcategory_combobox["values"] = subcategories
        self.subcategory_combobox["state"] = "readonly" if subcategories else "disabled"
        self.subcategory_combobox.set("")  # Limpa a seleção anterior de subcategoria
        self.start_button["state"] = "disabled" # Desabilita o botão de iniciar ao mudar a categoria
        self.subcategory_combobox.bind("<<ComboboxSelected>>", self.enable_start_button)

    def enable_start_button(self, event):
        if self.subcategory_combobox.get():
            self.start_button["state"] = "normal"
        else:
            self.start_button["ate"] = "disabled"

    def start_game(self):
        selected_category = self.category_combobox.get()
        selected_subcategory = self.subcategory_combobox.get()
        dict_options = self.vocabulary_model.get_next_word()
        if selected_category and selected_subcategory:
            print(f"Jogo iniciado com categoria: {selected_category}, subcategoria: {selected_subcategory}")
            # Passa as informações do jogo para a próxima página
            self.controller.mostrar_frame(GameVocabulary, game_category=selected_category, 
                                          game_subcategory=selected_subcategory,
                                          dict_options=dict_options)
        else:
            tk.messagebox.showinfo("Seleção Inválida", "Por favor, escolha uma categoria e subcategoria.")

class GameVocabulary(Pagina):
    def __init__(self, parent, controller):
        Pagina.__init__(self, parent, controller)
        self.controller = controller

    def load_game_content(self, category, subcategory, dict_options):
        """Carrega o conteúdo do jogo com base na categoria e subcategoria."""
        self.category = category
        self.subcategory = subcategory
        self.dict_options = dict_options
        label_info = tk.Label(self, text=f"Vocabulário - Categoria: {self.category}, Subcategoria: {self.subcategory}", font=("Arial", 14))
        

        self.centralizar_widget(label_info, 0.1)
        self.load_image_and_elements()

    def load_image_and_elements(self):
        # Carregando imagem (agora usando caminho relativo)
        # caminho_imagem = os.path.join("data", "options", "time_expressions", "april.png")
        caminho_imagem = r"C:\pyprojects\fast-english\app\extract_imgs\data\options\time_expressions\april.png"

        print("Dados>>> ", self.category, self.subcategory, self.dict_options)
        
        try:
            if not os.path.exists(caminho_imagem):
                raise FileNotFoundError(f"Arquivo de imagem '{caminho_imagem}' não encontrado.")
            imagem_pil = Image.open(caminho_imagem)
            imagem_tk = ImageTk.PhotoImage(imagem_pil)
            label_imagem = tk.Label(self, image=imagem_tk)
            label_imagem.image = imagem_tk
            self.centralizar_widget(label_imagem, 0.3)

            label_img_text = tk.Label(self, text="Abril")
            self.centralizar_widget(label_img_text, 0.4)

            self.answer_entry = tk.Entry(self, font=("Arial", 14))
            self.centralizar_widget(self.answer_entry, 0.5)

            botao_confirmar = tk.Button(self, text="Confirmar", command=self.verificar_resposta)
            botao_confirmar.place(relx=0.65, rely=0.7, anchor=tk.CENTER)

            botao_voltar_inicial = tk.Button(self, text="Voltar para Seleção", command=lambda: self.controller.mostrar_frame(PageGame))
            botao_voltar_inicial.place(relx=0.35, rely=0.7, anchor=tk.CENTER)

        except FileNotFoundError as e:
            mensagem_erro = tk.Label(self, text=str(e))
            self.centralizar_widget(mensagem_erro, 0.5)
        except Exception as e:
            mensagem_erro = tk.Label(self, text=f"Erro ao carregar a imagem: {e}")
            self.centralizar_widget(mensagem_erro, 0.5)

    def verificar_resposta(self):
        resposta = self.answer_entry.get()
        print(f"Categoria: {self.category}, Subcategoria: {self.subcategory}, Resposta do usuário: {resposta}")
        # Aqui você adicionaria a lógica para verificar a resposta

class ControladorDePaginas(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)

        self.title("Aplicação com Múltiplas Páginas")

        container = tk.Frame(self)
        container.place(relx=0, rely=0, relwidth=1, relheight=1)

        self.vocabulary_model = VocabularyModel()

        self.frames = {}
        for F in (PaginaInicial, PageGame, Pagina2, GameVocabulary):
            frame = F(container, self)
            self.frames[F] = frame
            frame.place(relx=0, rely=0, relwidth=1, relheight=1)

        self.mostrar_frame(PaginaInicial)

    def mostrar_frame(self, pagina_classe, **kwargs):
        frame = self.frames[pagina_classe]
        frame.tkraise()
        if pagina_classe == GameVocabulary and kwargs:
            frame.load_game_content(kwargs['game_category'], kwargs['game_subcategory'], kwargs['dict_options'])


    # def get_categories(self):
    #     vocabulary_model = VocabularyModel()
    #     return vocabulary_model.list_categories()
        # return ["Matemática", "Português", "Inglês", "História"]
        # vocabulary_model.list_subcategories('animals')

    # def get_subcategories(self, category):
    #     vocabulary_model = VocabularyModel()
    #     return vocabulary_model.list_subcategories(category)
    
    # def load_vocabulary(self, category, subcategory):
    #     vocabulary_model = VocabularyModel()

    #     return vocabulary_model.load_vocabulary(category=category, subcategory=subcategory)

    # def get_next_word(self):
    #     vocabulary_model = VocabularyModel()

    #     return vocabulary_model.get_next_word()

        
        # # Simulação da obtenção de subcategorias (substitua pela sua lógica real)
        # if category == "Matemática":
        #     return ["Adição", "Subtração", "Multiplicação", "Divisão"]
        # elif category == "Português":
        #     return ["Gramática", "Vocabulário", "Interpretação"]
        # elif category == "Inglês":
        #     return ["Verbos", "Gramática", "Vocabulário"]
        # elif category == "História":
        #     return ["Brasil", "Mundo", "Geral"]
        # else:
        #     return []
        
if __name__ == "__main__":
    app = ControladorDePaginas()
    app.geometry('600x400')
    app.mainloop()

AttributeError: '_tkinter.tkapp' object has no attribute 'vocabulary_model'

Português ['Gramática', 'Vocabulário', 'Interpretação'] . True ['Gramática', 'Vocabulário', 'Interpretação']
Jogo iniciado com categoria: Português, subcategoria: Vocabulário
Jogo iniciado com categoria: Português, subcategoria: Vocabulário
Categoria: Português, Subcategoria: Vocabulário, Resposta do usuário: 
Jogo iniciado com categoria: Português, subcategoria: Vocabulário


In [ ]:
# Isso é um w

import tkinter as tk

class Pagina(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller

class PaginaInicial(Pagina):
    def __init__(self, parent, controller):
        Pagina.__init__(self, parent, controller)
        label = tk.Label(self, text="Esta é a Página Inicial")
        label.pack(pady=10, padx=10)
        botao1 = tk.Button(self, text="Ir para a Página 1", command=lambda: controller.mostrar_frame(Pagina1))
        botao1.pack()
        botao2 = tk.Button(self, text="Ir para a Página 2", command=lambda: controller.mostrar_frame(Pagina2))
        botao2.pack()

class Pagina1(Pagina):
    def __init__(self, parent, controller):
        Pagina.__init__(self, parent, controller)
        label = tk.Label(self, text="Esta é a Página 1")
        label.pack(pady=10, padx=10)
        botao_voltar = tk.Button(self, text="Voltar para a Inicial", command=lambda: controller.mostrar_frame(PaginaInicial))
        botao_voltar.pack()

class Pagina2(Pagina):
    def __init__(self, parent, controller):
        Pagina.__init__(self, parent, controller)
        label = tk.Label(self, text="Esta é a Página 2")
        label.pack(pady=10, padx=10)
        botao_voltar = tk.Button(self, text="Voltar para a Inicial", command=lambda: controller.mostrar_frame(PaginaInicial))
        botao_voltar.pack()

class ControladorDePaginas(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)

        self.title("Aplicação com Múltiplas Páginas")

        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}
        for F in (PaginaInicial, Pagina1, Pagina2):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        self.mostrar_frame(PaginaInicial)

    def mostrar_frame(self, pagina_classe):
        frame = self.frames[pagina_classe]
        frame.tkraise() # Coloca o frame no topo da pilha de visualização

if __name__ == "__main__":
    app = ControladorDePaginas()
    app.geometry('600x400')
    app.mainloop()

----

In [20]:
# Python program to activate menu and toolbar
# with keyboard shortcut key

# Import the libraries tkinter and ttk
from tkinter import *
from tkinter import ttk

# Create a GUI app
app = Tk()

# Setting the title and geometry of the app
app.title('Vinayak App')
app.geometry('600x400')

# Creating function for menu bar
def menubar_shortcut(event = None):
	menubar = Menu()
	
	# Declare file and edit for showing in menu bar
	file = Menu(menubar, tearoff = False)
	edit = Menu(menubar, tearoff = False)

	# Display file and edit declared in previous step
	menubar.add_cascade(label = 'File', menu = file)
	menubar.add_cascade(label = 'Edit', menu = edit)

	# Display of menu bar in the app
	app.config(menu = menubar)

# Creating function for tool bar
def toolbar_shortcut(event = None):

	# Creating and displaying label for toolbar
	toolbar = ttk.Label(app)
	toolbar.pack(side = TOP, fill = X)

	# Creating and displaying of Bold button
	bold_btn = ttk.Button(toolbar, text = "Bold")
	bold_btn.grid(row=0, column = 0, padx = 5)

	# Creating and displaying of italic button
	italic_btn = ttk.Button(toolbar, text = "Italic")
	italic_btn.grid(row = 0, column = 1, padx = 5)

# Set up shortcut key for menu bar
app.bind('<Control-p>', menubar_shortcut)

# Set up shortcut key for tool bar
app.bind('<Control-q>', toolbar_shortcut)

# Make the loop for displaying app
app.mainloop()


In [11]:
# !pip install openpyxl

In [19]:
# Python program to illustrate the usage
# of hierarchical treeview in python GUI
# application using tkinter

# TODO: Inplementar na busca de arquivos e tlz

# Importing tkinter
from tkinter import *

# Importing ttk from tkinter
from tkinter import ttk 

# Creating app window
app = Tk() 

# Defining title of the app
app.title("GUI Application of Python") 

# Defining label of the app and calling a geometry
# management method i.e, pack in order to organize
# widgets in form of blocks before locating them 
# in the parent widget
ttk.Label(app, text ="Treeview(hierarchical)").pack()

# Creating treeview window
treeview = ttk.Treeview(app) 

# Calling pack method on the treeview
treeview.pack() 

# Inserting items to the treeview 
# Inserting parent
treeview.insert('', '0', 'item1', 
				text ='GeeksforGeeks') 

# Inserting child
treeview.insert('', '1', 'item2', 
				text ='Computer Science')
treeview.insert('', '2', 'item3', 
				text ='GATE papers')
treeview.insert('', 'end', 'item4',
				text ='Programming Languages')

# Inserting more than one attribute of an item
treeview.insert('item2', 'end', 'Algorithm', 
				text ='Algorithm') 
treeview.insert('item2', 'end', 'Data structure', 
				text ='Data structure') 
treeview.insert('item3', 'end', '2018 paper', 
				text ='2018 paper') 
treeview.insert('item3', 'end', '2019 paper', 
				text ='2019 paper')
treeview.insert('item4', 'end', 'Python', 
				text ='Python')
treeview.insert('item4', 'end', 'Java', 
				text ='Java')

# Placing each child items in parent widget
treeview.move('item2', 'item1', 'end') 
treeview.move('item3', 'item1', 'end')
treeview.move('item4', 'item1', 'end')

# Calling main() 
app.mainloop()


### Register file

In [17]:
# import openpyxl and tkinter modules
from openpyxl import *
from tkinter import *

# globally declare wb and sheet variable

# opening the existing excel file
wb = load_workbook('data/tabela_vazao_x_pressao_48h.xlsx')

# create the sheet object
sheet = wb.active


def excel():
    
    # resize the width of columns in
    # excel spreadsheet
    sheet.column_dimensions['A'].width = 30
    sheet.column_dimensions['B'].width = 10
    sheet.column_dimensions['C'].width = 10
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 20
    sheet.column_dimensions['F'].width = 40
    sheet.column_dimensions['G'].width = 50

    # write given data to an excel spreadsheet
    # at particular location
    sheet.cell(row=1, column=1).value = "Name"
    sheet.cell(row=1, column=2).value = "Course"
    sheet.cell(row=1, column=3).value = "Semester"
    sheet.cell(row=1, column=4).value = "Form Number"
    sheet.cell(row=1, column=5).value = "Contact Number"
    sheet.cell(row=1, column=6).value = "Email id"
    sheet.cell(row=1, column=7).value = "Address"


# Function to set focus (cursor)
def focus1(event):
    # set focus on the course_field box
    course_field.focus_set()


# Function to set focus
def focus2(event):
    # set focus on the sem_field box
    sem_field.focus_set()


# Function to set focus
def focus3(event):
    # set focus on the form_no_field box
    form_no_field.focus_set()


# Function to set focus
def focus4(event):
    # set focus on the contact_no_field box
    contact_no_field.focus_set()


# Function to set focus
def focus5(event):
    # set focus on the email_id_field box
    email_id_field.focus_set()


# Function to set focus
def focus6(event):
    # set focus on the address_field box
    address_field.focus_set()


# Function for clearing the
# contents of text entry boxes
def clear():
    
    # clear the content of text entry box
    name_field.delete(0, END)
    course_field.delete(0, END)
    sem_field.delete(0, END)
    form_no_field.delete(0, END)
    contact_no_field.delete(0, END)
    email_id_field.delete(0, END)
    address_field.delete(0, END)


# Function to take data from GUI 
# window and write to an excel file
def insert():
    
    # if user not fill any entry
    # then print "empty input"
    if (name_field.get() == "" and
        course_field.get() == "" and
        sem_field.get() == "" and
        form_no_field.get() == "" and
        contact_no_field.get() == "" and
        email_id_field.get() == "" and
        address_field.get() == ""):
            
        print("empty input")

    else:

        # assigning the max row and max column
        # value upto which data is written
        # in an excel sheet to the variable
        current_row = sheet.max_row
        current_column = sheet.max_column

        # get method returns current text
        # as string which we write into
        # excel spreadsheet at particular location
        sheet.cell(row=current_row + 1, column=1).value = name_field.get()
        sheet.cell(row=current_row + 1, column=2).value = course_field.get()
        sheet.cell(row=current_row + 1, column=3).value = sem_field.get()
        sheet.cell(row=current_row + 1, column=4).value = form_no_field.get()
        sheet.cell(row=current_row + 1, column=5).value = contact_no_field.get()
        sheet.cell(row=current_row + 1, column=6).value = email_id_field.get()
        sheet.cell(row=current_row + 1, column=7).value = address_field.get()

        # save the file
        wb.save('data/tabela_vazao_x_pressao_48h_2.xlsx')

        # set focus on the name_field box
        name_field.focus_set()

        # call the clear() function
        clear()


# Driver code
if __name__ == "__main__":
    
    # create a GUI window
    root = Tk()

    # set the background colour of GUI window
    root.configure(background='light green')

    # set the title of GUI window
    root.title("registration form")

    # set the configuration of GUI window
    root.geometry("500x300")

    excel()

    # create a Form label
    heading = Label(root, text="Form", bg="light green")

    # create a Name label
    name = Label(root, text="Name", bg="light green")

    # create a Course label
    course = Label(root, text="Course", bg="light green")

    # create a Semester label
    sem = Label(root, text="Semester", bg="light green")

    # create a Form No. label
    form_no = Label(root, text="Form No.", bg="light green")

    # create a Contact No. label
    contact_no = Label(root, text="Contact No.", bg="light green")

    # create a Email id label
    email_id = Label(root, text="Email id", bg="light green")

    # create a address label
    address = Label(root, text="Address", bg="light green")

    # grid method is used for placing
    # the widgets at respective positions
    # in table like structure .
    heading.grid(row=0, column=1)
    name.grid(row=1, column=0)
    course.grid(row=2, column=0)
    sem.grid(row=3, column=0)
    form_no.grid(row=4, column=0)
    contact_no.grid(row=5, column=0)
    email_id.grid(row=6, column=0)
    address.grid(row=7, column=0)

    # create a text entry box
    # for typing the information
    name_field = Entry(root)
    course_field = Entry(root)
    sem_field = Entry(root)
    form_no_field = Entry(root)
    contact_no_field = Entry(root)
    email_id_field = Entry(root)
    address_field = Entry(root)

    # bind method of widget is used for
    # the binding the function with the events

    # whenever the enter key is pressed
    # then call the focus1 function
    name_field.bind("<Return>", focus1)

    # whenever the enter key is pressed
    # then call the focus2 function
    course_field.bind("<Return>", focus2)

    # whenever the enter key is pressed
    # then call the focus3 function
    sem_field.bind("<Return>", focus3)

    # whenever the enter key is pressed
    # then call the focus4 function
    form_no_field.bind("<Return>", focus4)

    # whenever the enter key is pressed
    # then call the focus5 function
    contact_no_field.bind("<Return>", focus5)

    # whenever the enter key is pressed
    # then call the focus6 function
    email_id_field.bind("<Return>", focus6)

    # grid method is used for placing
    # the widgets at respective positions
    # in table like structure .
    name_field.grid(row=1, column=1, ipadx="100")
    course_field.grid(row=2, column=1, ipadx="100")
    sem_field.grid(row=3, column=1, ipadx="100")
    form_no_field.grid(row=4, column=1, ipadx="100")
    contact_no_field.grid(row=5, column=1, ipadx="100")
    email_id_field.grid(row=6, column=1, ipadx="100")
    address_field.grid(row=7, column=1, ipadx="100")

    # call excel function
    excel()

    # create a Submit Button and place into the root window
    submit = Button(root, text="Submit", fg="Black",
                            bg="Red", command=insert)
    submit.grid(row=8, column=1)

    # start the GUI
    root.mainloop()


### Color game

In [137]:
import tkinter
import random

class ColorGame(tkinter.Frame):
    def __init__(self, parent=None, **kwargs):
        tkinter.Frame.__init__(self, parent, **kwargs)
        self.parent = parent
        self.colours = ['Red', 'Blue', 'Green', 'Pink', 'Black',
                        'Yellow', 'Orange', 'White', 'Purple', 'Brown']
        self.score = 0
        self.timeleft = 30
        self.running = False  # Flag to indicate if the game is running

        self.instructions = tkinter.Label(self, text="Type in the colour of the words, and not the word text!",
                                         font=('Helvetica', 12))
        self.instructions.pack()

        self.scoreLabel = tkinter.Label(self, text="Press enter to start",
                                        font=('Helvetica', 12))
        self.scoreLabel.pack()

        self.timeLabel = tkinter.Label(self, text="Time left: " + str(self.timeleft),
                                      font=('Helvetica', 12))
        self.timeLabel.pack()

        self.label = tkinter.Label(self, font=('Helvetica', 60))
        self.label.pack()

        self.entry = tkinter.Entry(self)
        self.entry.pack()
        self.entry.focus_set()
        self.entry.bind('<Return>', self.startGame)

        self.nextColour() # Initial call to set up the first color

    def startGame(self, event):
        if not self.running:
            self.running = True
            self.score = 0
            self.timeleft = 30
            self.scoreLabel.config(text="Score: " + str(self.score))
            self.timeLabel.config(text="Time left: " + str(self.timeleft))
            self.countdown()
        self.nextColour()

    def nextColour(self):
        if self.running and self.timeleft > 0:
            self.entry.focus_set()
            if self.entry.get().lower() == self.colours[1].lower():
                self.score += 1
                self.scoreLabel.config(text="Score: " + str(self.score))
            self.entry.delete(0, tkinter.END)
            random.shuffle(self.colours)
            self.label.config(fg=str(self.colours[1]), text=str(self.colours[0]))

    def countdown(self):
        if self.running and self.timeleft > 0:
            self.timeleft -= 1
            self.timeLabel.config(text="Time left: " + str(self.timeleft))
            self.after(1000, self.countdown)
        elif self.running:
            self.running = False
            self.scoreLabel.config(text="Game Over! Final Score: " + str(self.score))

if __name__ == '__main__':
    root = tkinter.Tk()
    root.title("COLORGAME")
    game = ColorGame(root)
    game.pack(expand=True, fill='both')
    root.geometry("375x200")
    root.mainloop()

### Notepad

In [27]:
import tkinter
import os 
from tkinter import *
from tkinter.messagebox import *
from tkinter.filedialog import *

class Notepad:

	__root = Tk()

	# default window width and height
	__thisWidth = 300
	__thisHeight = 300
	__thisTextArea = Text(__root)
	__thisMenuBar = Menu(__root)
	__thisFileMenu = Menu(__thisMenuBar, tearoff=0)
	__thisEditMenu = Menu(__thisMenuBar, tearoff=0)
	__thisHelpMenu = Menu(__thisMenuBar, tearoff=0)
	
	# To add scrollbar
	__thisScrollBar = Scrollbar(__thisTextArea)	 
	__file = None

	def __init__(self,**kwargs):

		# Set icon
		try:
				self.__root.wm_iconbitmap("Notepad.ico") 
		except:
				pass

		# Set window size (the default is 300x300)

		try:
			self.__thisWidth = kwargs['width']
		except KeyError:
			pass

		try:
			self.__thisHeight = kwargs['height']
		except KeyError:
			pass

		# Set the window text
		self.__root.title("Untitled - Notepad")

		# Center the window
		screenWidth = self.__root.winfo_screenwidth()
		screenHeight = self.__root.winfo_screenheight()
	
		# For left-align
		left = (screenWidth / 2) - (self.__thisWidth / 2) 
		
		# For right-align
		top = (screenHeight / 2) - (self.__thisHeight /2) 
		
		# For top and bottom
		self.__root.geometry('%dx%d+%d+%d' % (self.__thisWidth,
											self.__thisHeight,
											left, top)) 

		# To make the textarea auto resizable
		self.__root.grid_rowconfigure(0, weight=1)
		self.__root.grid_columnconfigure(0, weight=1)

		# Add controls (widget)
		self.__thisTextArea.grid(sticky = N + E + S + W)
		
		# To open new file
		self.__thisFileMenu.add_command(label="New",
										command=self.__newFile) 
		
		# To open a already existing file
		self.__thisFileMenu.add_command(label="Open",
										command=self.__openFile)
		
		# To save current file
		self.__thisFileMenu.add_command(label="Save",
										command=self.__saveFile) 

		# To create a line in the dialog	 
		self.__thisFileMenu.add_separator()										 
		self.__thisFileMenu.add_command(label="Exit",
										command=self.__quitApplication)
		self.__thisMenuBar.add_cascade(label="File",
									menu=self.__thisFileMenu)	 
		
		# To give a feature of cut 
		self.__thisEditMenu.add_command(label="Cut",
										command=self.__cut)			 
	
		# to give a feature of copy 
		self.__thisEditMenu.add_command(label="Copy",
										command=self.__copy)		 
		
		# To give a feature of paste
		self.__thisEditMenu.add_command(label="Paste",
										command=self.__paste)		 
		
		# To give a feature of editing
		self.__thisMenuBar.add_cascade(label="Edit",
									menu=self.__thisEditMenu)	 
		
		# To create a feature of description of the notepad
		self.__thisHelpMenu.add_command(label="About Notepad",
										command=self.__showAbout) 
		self.__thisMenuBar.add_cascade(label="Help",
									menu=self.__thisHelpMenu)

		self.__root.config(menu=self.__thisMenuBar)

		self.__thisScrollBar.pack(side=RIGHT,fill=Y)				 
		
		# Scrollbar will adjust automatically according to the content	 
		self.__thisScrollBar.config(command=self.__thisTextArea.yview)	 
		self.__thisTextArea.config(yscrollcommand=self.__thisScrollBar.set)
	
		
	def __quitApplication(self):
		self.__root.destroy()
		# exit()

	def __showAbout(self):
		showinfo("Notepad","Mrinal Verma")

	def __openFile(self):
		
		self.__file = askopenfilename(defaultextension=".txt",
									filetypes=[("All Files","*.*"),
										("Text Documents","*.txt")])

		if self.__file == "":
			
			# no file to open
			self.__file = None
		else:
			
			# Try to open the file
			# set the window title
			self.__root.title(os.path.basename(self.__file) + " - Notepad")
			self.__thisTextArea.delete(1.0,END)

			file = open(self.__file,"r")

			self.__thisTextArea.insert(1.0,file.read())

			file.close()

		
	def __newFile(self):
		self.__root.title("Untitled - Notepad")
		self.__file = None
		self.__thisTextArea.delete(1.0,END)

	def __saveFile(self):

		if self.__file == None:
			# Save as new file
			self.__file = asksaveasfilename(initialfile='Untitled.txt',
											defaultextension=".txt",
											filetypes=[("All Files","*.*"),
												("Text Documents","*.txt")])

			if self.__file == "":
				self.__file = None
			else:
				
				# Try to save the file
				file = open(self.__file,"w")
				file.write(self.__thisTextArea.get(1.0,END))
				file.close()
				
				# Change the window title
				self.__root.title(os.path.basename(self.__file) + " - Notepad")
				
			
		else:
			file = open(self.__file,"w")
			file.write(self.__thisTextArea.get(1.0,END))
			file.close()

	def __cut(self):
		self.__thisTextArea.event_generate("<<Cut>>")

	def __copy(self):
		self.__thisTextArea.event_generate("<<Copy>>")

	def __paste(self):
		self.__thisTextArea.event_generate("<<Paste>>")

	def run(self):

		# Run main application
		self.__root.mainloop()




# Run main application
notepad = Notepad(width=600,height=400)
notepad.run()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Concremat\AppData\Local\Temp\ipykernel_41104\1844119945.py", line 143, in __openFile
    file = open(self.__file,"r")
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\pyprojects\fast-english\.venv\Lib\site-packages\openpyxl\reader\excel.py", line 346, in load_workbook
    reader = ExcelReader(filename, read_only, keep_vba,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\pyprojects\fast-english\.venv\Lib\site-packages\openpyxl\reader\excel.py", line 123, in __init__
    self.archive = _validate_archive(fn)
                   ^^^^^^^^^^^^^^^^^^^^^
  File "c:\pyprojects\fast-english\.venv\Lib\site-packages\openpyxl\reader\excel.py", line 93, in _validate_archive
    raise InvalidFileExcep